In [ ]:
"""
Cleaning and Linking SLIVER Laser + OGRE 

This script processes and links the SLIVER laser range data with positional
OGRE data. It cleans, parses, and merges the datasets and generates vizualizations 
of the map-view and along-track data. 

Data are available at https://conus.summitcamp.org/mirror/summit/ftp/science/ICESat/ICESat_data/
and OGRE .pos files are easily generated in Emlid Studio! 

If there are multiple .SCK files for the traverse, just use the function parse_laser_data for 
all of them, and then concatenate! 

If there are chunks of missing data (as there might be in the case of multiple .SCK files),
use the cumulative distance step cautiously... reference the map view! You'll want to look 
at the periods of continuous data along-track.

"""

In [ ]:
# Import necessary libraries 

import pandas as pd 
import numpy as np 
import datetime
import matplotlib.pyplot as plt 
from math import radians, sin, cos, atan2, sqrt 

In [ ]:
# Load OGRE .pos file 
ogre_pos = pd.read_csv( 
    './path/to/OGRE/posfile.pos', 
    skiprows=10, 
    delim_whitespace=10, 
    header=None
) 

# Assign column names to .pos 
ogre_pos.columns = [ 
    'Date', 'Time', 'Latitude', 'Longitude', 'Height', 
    'Q', 'ns', 'sdn', 'sde', 'sdu', 'sdne', 'sdeu', 
    'sdun', 'age', 'ratio'
] 

# Combine date and time columns into a single timestamp 
ogre_pos['timestamp'] = pd.to_datetime( 
    ogre_pos['Date'] + ' ' + ogre_pos['Time'],
    format='%Y/%m/%d %H:%M:%S.%f'
) 

In [ ]:
def parse_laser_data(filepath): 
    """
    Function to parse laser range data from a file, skipping lines with 
    corrupted formats and handling errors. 

    Parameters: 
    - filepath (str): Path to laser data file (.SCK)

    Returns: 
    - pd.DataFrame: DF with parsed timestamps and range values
    """

    data = [] # Empty list to hold parsed data 

    with open(filepath, 'r') as file: 
        for line_number, line in enumerate(file): 
            # Skip lines with GPS fixes or headers 
            if line.startswith("#") or line.startswith("$GPRMC") or not line.strip():
                continue

            parts = line.strip().split() # Split the line into parts 

            # Check if the line has the expected number of entries 
            if len(parts) < 7:
                print(f"[Line {line_number}] Skipping line due to unexpected format: {line}")
                continue 

            try: 
                # Parse date and time components if corrects entries are present in the line 
                day, month, year = int(parts[0]), int(parts[1]), int(parts[2])
                hour, minute, second = int(parts[3]), int(parts[4]), float(parts[5])
                timestamp = datetime.datetime(year + 2000, month, day, hour, minute, int(second), int((second % 1) * 1e6))
                
                # Parse range value 
                range_value = float(parts[6])
                data.append([timestamp, range_value])
            except (ValueError, IndexError) as e:
                # Handle parsing errors 
                print(f"[Line {line_number}] Error parsing line: {line}. Error: {e}")
                continue

    # Create a DataFrame from the parsed data 
    laser_df = pd.DataFrame(data, columns=["timestamp", "range"])
    laser_df['timestamp'] = pd.to_datetime(laser_df['timestamp'], errors='coerce')
    return laser_df  

In [ ]:
# Parse the SICK laser data 
sck = parse_laser_data('./path/to/SICK/file.SCK')

# If you have multiple SICK files, concatenate into one with pd.concat: 
# Ex: sck = pd.concat([sck0, sck1, sck2])

In [ ]:
# Merge the laser and positional data based on the nearest timestamp 
df = pd.merge_asof(
    sck.sort_values('timestamp'), 
    ogre_pos.sort_values('timestamp'), 
    left_on='timestamp', 
    right_on='timestamp', 
    direction='nearest'
)

# Calculate true surface elevation in centimeters 
df['elevation'] = (df['Height'] - df['range']) * 100

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Function using the Haversine formula to calculate the distance
    between two points. 

    Parameters: 
    - lat1, lon1, lat2, lon2 (float): Latitude and longitude (degrees) of the two points 

    Returns: 
    - float: Distance between the two points in meters 
    """

    R = 6371000 # Earth's radius in meters 
    lat1 = radians(lat1) # Convert degrees to radians 
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlat = lat2 - lat1 # Difference in lats and lons 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 # Haversine formula 
    c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
    distance = R * c # Distance in meters 
    return distance 

In [ ]:
# Compute cumulative distances alnog the track 
cumulative_distance = 0.0
previous_point = None
cumulative_distances = [] # List to store cumulative distances 

for index, row in df.iterrows():
    if previous_point is not None:
        # Calculate the distance between consecutive points 
        distance = haversine(previous_point['Latitude'], previous_point['Longitude'], row['Latitude'], row['Longitude'])
        cumulative_distance += distance # Update cumulative distance 
    else:
        # Initialize for the first point 
        distance = 0.0
        cumulative_distance = 0.0
    cumulative_distances.append(cumulative_distance) # Append cumulative distances 
    previous_point = row # Update the previous point 

df['Distance'] = cumulative_distances  # Add cumulative distances to the DataFrame 

In [ ]:
# Plot a map view of the traverse lat/lons 
plt.figure()
plt.scatter(df['Longitude'], df['Latitude'], s=1)
plt.title('Map-view plot of ICESat Traverse')
plt.xlabel('Longitude(Deg)')
plt.ylabel('Latitude(Deg)')
plt.show()

In [ ]:
# Plot along-track elevation data 
plt.figure()
plt.scatter(df['Distance'], df['range'], s=1)
plt.title('Elevations Along-Track')
plt.xlabel('Distance Along-Track [m]')
plt.ylabel('Distance From Laser [m]')
plt.show()